In [ ]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install sentence_transformers

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ast

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the SentenceTransformer model and move it to the GPU
model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to(device)

# Step 1: Load input sentences and entity knowledge strings from CSVs
input_df = pd.read_csv('/kaggle/input/business-json/Task_3_data.csv')  # CSV with columns 'sentence' and 'actual_entity'
entity_df = pd.read_csv('/kaggle/input/business-json/Knowledge.csv')  # CSV with columns 'entity_name' and 'entity_description'

# # Function to extract entity names
# def extract_entity_name(entities_str):
#     entities_list = ast.literal_eval(entities_str)
#     # Return the first two mentions if they exist, otherwise None
#     return entities_list

# # Apply the function to the 'entities' column and expand into two new columns
# input_df['entity_names'] = input_df['entity_titles'].apply(extract_entity_name).apply(pd.Series)

# Display the updated DataFrame
print(input_df)

asr_df = pd.read_csv('/kaggle/input/business-json/task_3_asr_out.csv')

In [ ]:
print(input_df.dtypes)

# Top-K entity Linking

In [ ]:
NER_df = pd.read_csv('/kaggle/input/business-json/Task_3_asr_NER.csv')

In [ ]:
question_embeddings = model.encode(input_df['question'].tolist(), convert_to_tensor=True, device=device)
entity_knowledge_embeddings = model.encode(entity_df['Knowledge'].tolist(), convert_to_tensor=True, device=device)
# all_embeddings = [] 

# for index, row in asr_df.iterrows():
#     sentences_str = row['transcriptions'] 
#     sentences = ast.literal_eval(sentences_str)
#     row_embeddings = model.encode(sentences, convert_to_tensor=True, device=device)
#     all_embeddings.append(row_embeddings)  # Append the embeddings to the list


## After NER

### Full Knowledge

In [ ]:
def calc(result_df, k, thresh):
    f1_s = []
    r_s = []
    p_s = []
    for i, row in result_df.iterrows():
        count = row['bool_c']
        act = row['bool_t']
        
        l = len(row['retrieved_sentence_ids'])
        if (l == 0):
            f1 = 0
            recall = 0
            pr = 0
        else:
            pr = count/l
            recall = count/act
            if (recall == 0 and pr == 0):
                f1 = 0
            else:
                f1 = 2 * pr * recall/(pr+recall)

        f1_s.append(f1)    
        r_s.append(recall)
        p_s.append(pr)

#     print(f'average f1 score for top-{k} reults with threshold = {thresh} : {(sum(f1_s)/len(f1_s))}')
#     print(f'average recall score for top-{k} reults with threshold = {thresh} : {(sum(r_s)/len(r_s))}')
#     print(f'average precision score for top-{k} reults with threshold = {thresh} : {(sum(p_s)/len(p_s))}')
    return (sum(f1_s)/len(f1_s)), (sum(r_s)/len(r_s)), (sum(p_s)/len(p_s))

def calc_ner(result_df, k, thresh):
    f1_s = []
    r_s = []
    p_s = []
    for i, row in result_df.iterrows():
        count = row['bool_c']
        act = row['bool_t']
        
        l = len(row['retrieved_sentence_ids'])
        if (l == 0):
            f1 = 0
            recall = 0
            pr = 0
        else:
            pr = count/l
            recall = count/act
            if (recall == 0 and pr == 0):
                f1 = 0
            else:
                f1 = 2 * pr * recall/(pr+recall)

        f1_s.append(f1)    
        r_s.append(recall)
        p_s.append(pr)

#     print(f'average f1 score for top-{k} reults with threshold = {thresh} : {(sum(f1_s)/len(f1_s))}')
#     print(f'average recall score for top-{k} reults with threshold = {thresh} : {(sum(r_s)/len(r_s))}')
#     print(f'average precision score for top-{k} reults with threshold = {thresh} : {(sum(p_s)/len(p_s))}')
    return (sum(f1_s)/len(f1_s)), (sum(r_s)/len(r_s)), (sum(p_s)/len(p_s))

# Assume `knowledge_embeddings` contains embeddings of all entities
# Assume `knowledge_entity_ids` contains the IDs corresponding to the knowledge embeddings
dc = {}
for z in range(2,3):
    for so in range(1,2):
        top_k = 20  # Define how many top entities you want to retrieve
        similarity_threshold = 0.25 * so  # Define the similarity threshold
        similarity_threshold = round(similarity_threshold,2)
        result_data = []

        for i, row in input_df.iterrows():
            # Step 1: Get the sentence embeddings for the current row from the asr_df
            question_embedding = question_embeddings[i].unsqueeze(0).cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)

            # Step 2: Get the corresponding sentence IDs for this row
            sentence_ids = ast.literal_eval(asr_df['Sentence_ids'].iloc[i])

            entity_ids = ast.literal_eval(NER_df['top_entities'].iloc[i])
            sim_scores = []
            emt = []
            for j in range(len(sentence_ids)):
                # Get similarity scores for the current sentence
        #         sentence_similarity_scores = similarity_scores[j]
                sentence_id = sentence_ids[j]
                entity_id = entity_ids[j]
                emt.append(entity_id)
                ind = entity_df[entity_df['Entity_ID'] == entity_id].index
                embed = entity_knowledge_embeddings[ind]
                similarity_scores = cosine_similarity(question_embedding,  embed.cpu().numpy()).flatten() 
                sim_scores.append(similarity_scores)
                # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sim_scores) if score >= similarity_threshold]
            top_indices = sorted(filtered_indices, key=lambda idx: sim_scores[idx], reverse=True)[:top_k]
            pos_sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])
#             emt = emt[top_indices]
            count = 0
            for e1 in pos_sentence_ids:
                if e1 in top_indices:
                    count+=1
            result_data.append({
                'q_id': row['QID'],
                'sentence_ids': sentence_ids,
                'retrieved_sentence_ids': top_indices,
                'retrieved_sentence_entity_ids': [emt[idx] for idx in top_indices],
                'similarity_scores': [sim_scores[idx][0] for idx in top_indices],
                'ground_truth_positive_sentence_ids': pos_sentence_ids,
                'ground_truth_positive_sentence_entity_ids': ast.literal_eval(row['entities']),
                'all_sentences': ast.literal_eval(row['all_sentences']),
                'answer_num': len(pos_sentence_ids),
                'answer':row['answer'],
                'bool_c' : count,
                'bool_t': len(pos_sentence_ids)
            })
        #         # Sort by similarity score and get top k entities
        #     top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]
        # #         if i < 1: 
        # #             print(top_indice)
        #         # Get the top entities and their corresponding IDs
        #     top_entities_for_sentence = knowledge_entity_ids[top_indice]
        #         top_scores_for_sentence = [sentence_similarity_scores[idx] for idx in top_indices]

                # Store results for this sentence
        #         sentence_ids.append(sentence_ids[j])

            # List of sentence IDs corresponding to the embeddings

        #     # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
        #     similarity_scores = cosine_similarity(question_embedding,  entity_knowledge_embeddings.cpu().numpy()).flatten()  # Shape: (embedding_dim, num_sentences_in_row)

        #     filtered_indices = [idx for idx, score in enumerate(similarity_scores) if score >= similarity_threshold]

        # #     # Step 6: Sort by similarity score and get top k sentences
        #     top_indices = sorted(filtered_indices, key=lambda idx: similarity_scores[idx], reverse=True)[:top_k]
        #     top_entities = #         top_scores_for_sentence = [sentence_similarity_scores[idx] for idx in top_indices]
        #     # Get the top k sentences and their corresponding sentence IDs
        #     top_sentence_ids = [sentence_ids[idx] for idx in top_indices]  # Corresponding sentence IDs
        #     top_sentences = [ast.literal_eval(asr_df['transcriptions'].iloc[i])[idx] for idx in top_indices]  # Sentences corresponding to top indices
        #     pos_sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])
        #     count = 0
        #     for e1 in pos_sentence_ids:
        #         if e1 in top_sentence_ids:
        #             count+=1
        # #     print(pos_sentence_ids.all() in top_sentence_ids)
        #     # Step 7: Store the result for this row
        #     result_data.append({
        #         'q_id': row['QID'],
        #         'top_sentence_ids': top_sentence_ids,
        #         'top_sentences': top_sentences,
        #         'similarity_scores': [similarity_scores[idx] for idx in top_indices],

        #     })
            # Step 4: Process each sentence
        #     top_sentence_ids = []
        #     top_entities = []
        #     top_scores = []

        #     for j in range(len(sentence_ids)):
        #         # Get similarity scores for the current sentence
        #         sentence_similarity_scores = similarity_scores[j]

        #         # Filter based on similarity threshold
        #         filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]

        #         # Sort by similarity score and get top k entities
        #         top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]
        # #         if i < 1: 
        # #             print(top_indice)
        #         # Get the top entities and their corresponding IDs
        #         top_entities_for_sentence = knowledge_entity_ids[top_indice]
        # #         top_scores_for_sentence = [sentence_similarity_scores[idx] for idx in top_indices]

        #         # Store results for this sentence
        # #         sentence_ids.append(sentence_ids[j])
        #         top_entities.append(top_entities_for_sentence)
        # #         top_scores.append(top_scores_for_sentence)

        #     # Flatten the results and count matches for positive entities
        # #     pos_sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])
        # #     count = sum(e1 in top_entity for top_entities_for_sentence in top_entities for e1 in pos_sentence_ids)

        #     # Step 7: Store the result for this row


        # Optionally convert the result to a DataFrame
        result_df = pd.DataFrame(result_data)
        a, b, c = calc(result_df, top_k, similarity_threshold)
        dc[top_k, similarity_threshold] = a,b,c
        ojjj = 0
        for i , row in result_df.iterrows():
#                 kjsh = ast.literal_eval(row['top_sentence_ids'])
            kjsh = row['retrieved_sentence_ids']
            ojjj += len(kjsh)
        print(f'For threshold = {similarity_threshold}: average_result_length = {ojjj/len(result_df)}')
# Print the result
# print(result_df.head())


In [ ]:
result_df.to_csv("Task3_Retrieved_results.csv")

In [ ]:
for it in dc.items():
    print(it)

#### NER after retrieval with full knowledge

In [ ]:
new_df = pd.read_csv("/kaggle/working/Task3_Retrieved_results.csv")
entity_embeddings = model.encode(entity_df['Knowledge'].tolist(), convert_to_tensor=True, device=device)
knowledge_entity_ids = entity_df['Entity_ID'].tolist()

In [ ]:
all_embeddings = [] 

for index, row in new_df.iterrows():
    sentences_ids_str = row['retrieved_sentence_ids'] 
    sentence_ids = ast.literal_eval(sentences_ids_str)
    sentences_str = asr_df['transcriptions'].iloc[index] 
    sentences = ast.literal_eval(sentences_str)
    relevant = []
    for j in range(len(sentences)):
        if j in sentence_ids:
            relevant.append(sentences[j])
    row_embeddings = model.encode(relevant, convert_to_tensor=True, device=device)
    all_embeddings.append(row_embeddings)  # Append the embeddings to the list

In [ ]:
result_data = []

In [ ]:
similarity_threshold = 0.05

# Create a dictionary to store results, ensuring unique q_id entries
result_data = {}

for i, row in new_df.iterrows():
    # Step 1: Get the sentence embeddings for the current row from the asr_df
    sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
    if len(sentence_embeddings) > 0:
        # Step 2: Get the corresponding sentence IDs for this row
        sentence_ids = ast.literal_eval(new_df['retrieved_sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings

        # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
        similarity_scores = cosine_similarity(sentence_embeddings, entity_embeddings.cpu().numpy())  # Shape: (embedding_dim, num_sentences_in_row)

        # Step 4: Process each sentence
        top_entities = []
        
        for j in range(len(sentence_ids)):
            # Get similarity scores for the current sentence
            sentence_similarity_scores = similarity_scores[j]

            # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]
            if filtered_indices:
                # Sort by similarity score and get top entity
                top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]

                # Get the top entity for the sentence
                top_entities_for_sentence = knowledge_entity_ids[top_indice]
                top_entities.append(top_entities_for_sentence)

    else:
        top_entities = []
        sentence_ids = []
    # Step 7: Store the result for this q_id
    if row['q_id'] not in result_data:
        result_data[row['q_id']] = {
            'q_id': row['q_id'],
            'retrieved_sentence_ids': sentence_ids,
            'top_entities': top_entities
        }

# Convert the result dictionary to a DataFrame
result_df = pd.DataFrame(list(result_data.values()))

# Print the result
print(result_df['q_id'].value_counts())

In [ ]:
result_df.to_csv("Task3_CLIP_full_Knowledge_NER_results.csv")

#### NER after retrieval with partial knowledge

In [ ]:
entity_df['Knowledge_20'] = entity_df['Knowledge'].apply(lambda x: x[:int(len(x) * 0.2)])
entity_embeddings = model.encode(entity_df['Knowledge_20'].tolist(), convert_to_tensor=True, device=device)

In [ ]:
similarity_threshold = 0.05

# Create a dictionary to store results, ensuring unique q_id entries
result_data = {}

for i, row in new_df.iterrows():
    # Step 1: Get the sentence embeddings for the current row from the asr_df
    sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
    if len(sentence_embeddings) > 0:
        # Step 2: Get the corresponding sentence IDs for this row
        sentence_ids = ast.literal_eval(new_df['retrieved_sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings

        # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
        similarity_scores = cosine_similarity(sentence_embeddings, entity_embeddings.cpu().numpy())  # Shape: (embedding_dim, num_sentences_in_row)

        # Step 4: Process each sentence
        top_entities = []
        
        for j in range(len(sentence_ids)):
            # Get similarity scores for the current sentence
            sentence_similarity_scores = similarity_scores[j]

            # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]
            if filtered_indices:
                # Sort by similarity score and get top entity
                top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]

                # Get the top entity for the sentence
                top_entities_for_sentence = knowledge_entity_ids[top_indice]
                top_entities.append(top_entities_for_sentence)

    else:
        top_entities = []
        sentence_ids = []
    # Step 7: Store the result for this q_id
    if row['q_id'] not in result_data:
        result_data[row['q_id']] = {
            'q_id': row['q_id'],
            'retrieved_sentence_ids': sentence_ids,
            'top_entities': top_entities
        }

# Convert the result dictionary to a DataFrame
result_df = pd.DataFrame(list(result_data.values()))

# Print the result
print(result_df['q_id'].value_counts())

In [ ]:
result_df.to_csv("Task3_CLIP_partial_Knowledge_NER_results.csv")

#### NER after retrieval with label (Entity_names)

In [ ]:
entity_embeddings = model.encode(entity_df['Entity_name'].tolist(), convert_to_tensor=True, device=device)

In [ ]:
similarity_threshold = 0.05

# Create a dictionary to store results, ensuring unique q_id entries
result_data = {}

for i, row in new_df.iterrows():
    # Step 1: Get the sentence embeddings for the current row from the asr_df
    sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
    if len(sentence_embeddings) > 0:
        # Step 2: Get the corresponding sentence IDs for this row
        sentence_ids = ast.literal_eval(new_df['retrieved_sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings

        # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
        similarity_scores = cosine_similarity(sentence_embeddings, entity_embeddings.cpu().numpy())  # Shape: (embedding_dim, num_sentences_in_row)

        # Step 4: Process each sentence
        top_entities = []
        
        for j in range(len(sentence_ids)):
            # Get similarity scores for the current sentence
            sentence_similarity_scores = similarity_scores[j]

            # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]
            if filtered_indices:
                # Sort by similarity score and get top entity
                top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]

                # Get the top entity for the sentence
                top_entities_for_sentence = knowledge_entity_ids[top_indice]
                top_entities.append(top_entities_for_sentence)

    else:
        top_entities = []
        sentence_ids = []
    # Step 7: Store the result for this q_id
    if row['q_id'] not in result_data:
        result_data[row['q_id']] = {
            'q_id': row['q_id'],
            'retrieved_sentence_ids': sentence_ids,
            'top_entities': top_entities
        }

# Convert the result dictionary to a DataFrame
result_df = pd.DataFrame(list(result_data.values()))

# Print the result
print(result_df['q_id'].value_counts())

In [ ]:
result_df.to_csv("Task3_CLIP_label_NER_results.csv")

#### NER on pos results with full_knowledge

In [ ]:
all_embeddings = [] 
entity_embeddings = model.encode(entity_df['Knowledge'].tolist(), convert_to_tensor=True, device=device)
for index, row in input_df.iterrows():
    sentences_ids_str = row['positive_sentence_ids'] 
    sentence_ids = ast.literal_eval(sentences_ids_str)
    sentences_str = asr_df['transcriptions'].iloc[index] 
    sentences = ast.literal_eval(sentences_str)
    relevant = []
    for j in range(len(sentences)):
        if j in sentence_ids:
            relevant.append(sentences[j])
    row_embeddings = model.encode(relevant, convert_to_tensor=True, device=device)
    all_embeddings.append(row_embeddings)  # Append the embeddings to the list

In [ ]:
similarity_threshold = 0.05

# Create a dictionary to store results, ensuring unique q_id entries
result_data = {}

for i, row in input_df.iterrows():
    # Step 1: Get the sentence embeddings for the current row from the asr_df
    sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
    if len(sentence_embeddings) > 0:
        # Step 2: Get the corresponding sentence IDs for this row
        sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings

        # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
        similarity_scores = cosine_similarity(sentence_embeddings, entity_embeddings.cpu().numpy())  # Shape: (embedding_dim, num_sentences_in_row)

        # Step 4: Process each sentence
        top_entities = []
        
        for j in range(len(sentence_ids)):
            # Get similarity scores for the current sentence
            sentence_similarity_scores = similarity_scores[j]

            # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]
            if filtered_indices:
                # Sort by similarity score and get top entity
                top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]

                # Get the top entity for the sentence
                top_entities_for_sentence = knowledge_entity_ids[top_indice]
                top_entities.append(top_entities_for_sentence)

    else:
        top_entities = []

    # Step 7: Store the result for this q_id
    if row['QID'] not in result_data:
        result_data[row['QID']] = {
            'q_id': row['QID'],
            'positive_sentence_ids': sentence_ids,
            'top_entities': top_entities
        }

# Convert the result dictionary to a DataFrame
result_df = pd.DataFrame(list(result_data.values()))

# Print the result
print(result_df['q_id'].value_counts())

In [ ]:
result_df.to_csv("Task3_CLIP_full_knowledge_positive_sentences_NER_results.csv")

#### NER on pos results with partial Knowledge

In [ ]:
entity_df['Knowledge_20'] = entity_df['Knowledge'].apply(lambda x: x[:int(len(x) * 0.2)])
entity_embeddings = model.encode(entity_df['Knowledge_20'].tolist(), convert_to_tensor=True, device=device)

In [ ]:
similarity_threshold = 0.05

# Create a dictionary to store results, ensuring unique q_id entries
result_data = {}

for i, row in input_df.iterrows():
    # Step 1: Get the sentence embeddings for the current row from the asr_df
    sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
    if len(sentence_embeddings) > 0:
        # Step 2: Get the corresponding sentence IDs for this row
        sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings

        # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
        similarity_scores = cosine_similarity(sentence_embeddings, entity_embeddings.cpu().numpy())  # Shape: (embedding_dim, num_sentences_in_row)

        # Step 4: Process each sentence
        top_entities = []
        
        for j in range(len(sentence_ids)):
            # Get similarity scores for the current sentence
            sentence_similarity_scores = similarity_scores[j]

            # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]
            if filtered_indices:
                # Sort by similarity score and get top entity
                top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]

                # Get the top entity for the sentence
                top_entities_for_sentence = knowledge_entity_ids[top_indice]
                top_entities.append(top_entities_for_sentence)

    else:
        top_entities = []

    # Step 7: Store the result for this q_id
    if row['QID'] not in result_data:
        result_data[row['QID']] = {
            'q_id': row['QID'],
            'positive_sentence_ids': sentence_ids,
            'top_entities': top_entities
        }

# Convert the result dictionary to a DataFrame
result_df = pd.DataFrame(list(result_data.values()))

# Print the result
print(result_df['q_id'].value_counts())

In [ ]:
result_df.to_csv("Task3_CLIP_partial_knowledge_positive_sentences_NER_results.csv")

#### NER on pos results with labels(entity_names)

In [ ]:
entity_embeddings = model.encode(entity_df['Entity_name'].tolist(), convert_to_tensor=True, device=device)

In [ ]:
similarity_threshold = 0.05

# Create a dictionary to store results, ensuring unique q_id entries
result_data = {}

for i, row in input_df.iterrows():
    # Step 1: Get the sentence embeddings for the current row from the asr_df
    sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
    if len(sentence_embeddings) > 0:
        # Step 2: Get the corresponding sentence IDs for this row
        sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings

        # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
        similarity_scores = cosine_similarity(sentence_embeddings, entity_embeddings.cpu().numpy())  # Shape: (embedding_dim, num_sentences_in_row)

        # Step 4: Process each sentence
        top_entities = []
        
        for j in range(len(sentence_ids)):
            # Get similarity scores for the current sentence
            sentence_similarity_scores = similarity_scores[j]

            # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]
            if filtered_indices:
                # Sort by similarity score and get top entity
                top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]

                # Get the top entity for the sentence
                top_entities_for_sentence = knowledge_entity_ids[top_indice]
                top_entities.append(top_entities_for_sentence)

    else:
        top_entities = []

    # Step 7: Store the result for this q_id
    if row['QID'] not in result_data:
        result_data[row['QID']] = {
            'q_id': row['QID'],
            'positive_sentence_ids': sentence_ids,
            'top_entities': top_entities
        }

# Convert the result dictionary to a DataFrame
result_df = pd.DataFrame(list(result_data.values()))

# Print the result
print(result_df['q_id'].value_counts())

In [ ]:
result_df.to_csv("Task3_CLIP_label_positive_sentences_NER_results.csv")

In [ ]:
# similarity_threshold = 0.05

# for i, row in new_df.iterrows():
#     # Step 1: Get the sentence embeddings for the current row from the asr_df
#     sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
#     if len(sentence_embeddings > 0):
#         # Step 2: Get the corresponding sentence IDs for this row
#         sentence_ids = ast.literal_eval(new_df['retrieved_sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings

#         # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
#         similarity_scores = cosine_similarity(sentence_embeddings,  entity_embeddings.cpu().numpy())  # Shape: (embedding_dim, num_sentences_in_row)

#     #     print(similarity_scores.shape)
#         # Step 4: Process each sentence
#     #     top_sentence_ids = []
#         top_entities = []
#         top_scores = []

#         for j in range(len(sentence_ids)):
#             # Get similarity scores for the current sentence
#             sentence_similarity_scores = similarity_scores[j]

#             # Filter based on similarity threshold
#             filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]
#             # Sort by similarity score and get top k entities
#             top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]
#     #         if i < 1: 
#     #             print(top_indice)
#             # Get the top entities and their corresponding IDs
#             top_entities_for_sentence = knowledge_entity_ids[top_indice]
#     #         top_scores_for_sentence = [sentence_similarity_scores[idx] for idx in top_indices]

#             # Store results for this sentence
#     #         sentence_ids.append(sentence_ids[j])
#             top_entities.append(top_entities_for_sentence)
#     #         top_scores.append(top_scores_for_sentence)

#         # Flatten the results and count matches for positive entities
#     #     pos_sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])
#     #     count = sum(e1 in top_entity for top_entities_for_sentence in top_entities for e1 in pos_sentence_ids)
#     else:
#         top_entities = []
#     # Step 7: Store the result for this row
#     result_data.append({
#         'q_id': row['q_id'],
#         'sentence_ids': sentence_ids,
#         'top_entities': top_entities
# #         'similarity_scores': top_scores,
# #         'bool_c': count,
# #         'bool_t': len(pos_sentence_ids)
#     })

# # Optionally convert the result to a DataFrame
# result_df = pd.DataFrame(result_data)

# # Print the result
# print(result_df['q_id'].value_counts())

In [ ]:
# result_df.to_csv("help.csv")

### Partial Knowledge

In [ ]:
entity_df['Knowledge_20'] = entity_df['Knowledge'].apply(lambda x: x[:int(len(x) * 0.2)])
question_embeddings = model.encode(input_df['question'].tolist(), convert_to_tensor=True, device=device)
entity_knowledge_embeddings = model.encode(entity_df['Knowledge_20'].tolist(), convert_to_tensor=True, device=device)

In [ ]:

# Assume `knowledge_embeddings` contains embeddings of all entities
# Assume `knowledge_entity_ids` contains the IDs corresponding to the knowledge embeddings

for z in range(2,3):
    for so in range(10,30):
        dc = {}
        top_k = 20  # Define how many top entities you want to retrieve
        similarity_threshold = 0.01 * so  # Define the similarity threshold
        similarity_threshold = round(similarity_threshold,2)
        result_data = []

        for i, row in input_df.iterrows():
            # Step 1: Get the sentence embeddings for the current row from the asr_df
            question_embedding = question_embeddings[i].unsqueeze(0).cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)

            # Step 2: Get the corresponding sentence IDs for this row
            sentence_ids = ast.literal_eval(asr_df['Sentence_ids'].iloc[i])

            entity_ids = ast.literal_eval(NER_df['top_entities'].iloc[i])
            sim_scores = []
            emt = []
            for j in range(len(sentence_ids)):
                # Get similarity scores for the current sentence
        #         sentence_similarity_scores = similarity_scores[j]
                sentence_id = sentence_ids[j]
                entity_id = entity_ids[j]
                emt.append(entity_id)
                ind = entity_df[entity_df['Entity_ID'] == entity_id].index
                embed = entity_knowledge_embeddings[ind]
                similarity_scores = cosine_similarity(question_embedding,  embed.cpu().numpy()).flatten() 
                sim_scores.append(similarity_scores)
                # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sim_scores) if score >= similarity_threshold]
            top_indices = sorted(filtered_indices, key=lambda idx: sim_scores[idx], reverse=True)[:top_k]
            pos_sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])
#             emt = emt[top_indices]
            count = 0
            for e1 in pos_sentence_ids:
                if e1 in top_indices:
                    count+=1
            result_data.append({
                'q_id': row['QID'],
                'sentence_ids': sentence_ids,
                'retrieved_sentence_ids': top_indices,
                'retrieved_sentence_entity_ids': [emt[idx] for idx in top_indices],
                'similarity_scores': [sim_scores[idx][0] for idx in top_indices],
                'ground_truth_positive_sentence_ids': pos_sentence_ids,
                'ground_truth_positive_sentence_entity_ids': ast.literal_eval(row['entities']),
                'all_sentences': ast.literal_eval(row['all_sentences']),
                'answer_num': len(pos_sentence_ids),
                'answer':'answer',
                'bool_c' : count,
                'bool_t': len(pos_sentence_ids)
            })

        # Optionally convert the result to a DataFrame
        result_df = pd.DataFrame(result_data)
        a, b, c = calc(result_df, top_k, similarity_threshold)
        dc[top_k, similarity_threshold] = a,b,c
        ojjj = 0
        for i , row in result_df.iterrows():
#                 kjsh = ast.literal_eval(row['top_sentence_ids'])
            kjsh = row['retrieved_sentence_ids']
            ojjj += len(kjsh)
        print(f'For threshold = {similarity_threshold}: average_result_length = {ojjj/len(result_df)}')
# Print the result
# print(result_df.head())
        for it in dc.items():
            print(it)

In [ ]:
for it in dc.items():
    print(it)

In [ ]:
result_df.to_csv("Task3_CLIP_partial_knowledge_Retrieved_results.csv")

### Entity Labels only

In [ ]:
question_embeddings = model.encode(input_df['question'].tolist(), convert_to_tensor=True, device=device)
entity_knowledge_embeddings = model.encode(entity_df['Entity_name'].tolist(), convert_to_tensor=True, device=device)

In [ ]:

# Assume `knowledge_embeddings` contains embeddings of all entities
# Assume `knowledge_entity_ids` contains the IDs corresponding to the knowledge embeddings

for z in range(2,3):
    for so in range(1,2):
        dc = {}
        top_k = 20  # Define how many top entities you want to retrieve
        similarity_threshold = 0.15 * so  # Define the similarity threshold
        similarity_threshold = round(similarity_threshold,2)
        result_data = []

        for i, row in input_df.iterrows():
            # Step 1: Get the sentence embeddings for the current row from the asr_df
            question_embedding = question_embeddings[i].unsqueeze(0).cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)

            # Step 2: Get the corresponding sentence IDs for this row
            sentence_ids = ast.literal_eval(asr_df['Sentence_ids'].iloc[i])

            entity_ids = ast.literal_eval(NER_df['top_entities'].iloc[i])
            sim_scores = []
            emt = []
            for j in range(len(sentence_ids)):
                # Get similarity scores for the current sentence
        #         sentence_similarity_scores = similarity_scores[j]
                sentence_id = sentence_ids[j]
                entity_id = entity_ids[j]
                emt.append(entity_id)
                ind = entity_df[entity_df['Entity_ID'] == entity_id].index
                embed = entity_knowledge_embeddings[ind]
                similarity_scores = cosine_similarity(question_embedding,  embed.cpu().numpy()).flatten() 
                sim_scores.append(similarity_scores)
                # Filter based on similarity threshold
            filtered_indices = [idx for idx, score in enumerate(sim_scores) if score >= similarity_threshold]
            top_indices = sorted(filtered_indices, key=lambda idx: sim_scores[idx], reverse=True)[:top_k]
            pos_sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])
#             emt = emt[top_indices]
            count = 0
            for e1 in pos_sentence_ids:
                if e1 in top_indices:
                    count+=1
            result_data.append({
                'q_id': row['QID'],
                'sentence_ids': sentence_ids,
                'retrieved_sentence_ids': top_indices,
                'retrieved_sentence_entity_ids': [emt[idx] for idx in top_indices],
                'similarity_scores': [sim_scores[idx][0] for idx in top_indices],
                'ground_truth_positive_sentence_ids': pos_sentence_ids,
                'ground_truth_positive_sentence_entity_ids': ast.literal_eval(row['entities']),
                'all_sentences': ast.literal_eval(row['all_sentences']),
                'answer_num': len(pos_sentence_ids),
                'answer':'answer',
                'bool_c' : count,
                'bool_t': len(pos_sentence_ids)
            })

        # Optionally convert the result to a DataFrame
        result_df = pd.DataFrame(result_data)
        a, b, c = calc(result_df, top_k, similarity_threshold)
        dc[top_k, similarity_threshold] = a,b,c
        ojjj = 0
        for i , row in result_df.iterrows():
#                 kjsh = ast.literal_eval(row['top_sentence_ids'])
            kjsh = row['retrieved_sentence_ids']
            ojjj += len(kjsh)
        print(f'For threshold = {similarity_threshold}: average_result_length = {ojjj/len(result_df)}')
# Print the result
# print(result_df.head())
        for it in dc.items():
            print(it)

In [ ]:
result_df.to_csv("Task3_CLIP_label_Retrieved_results.csv")

In [ ]:

# for i in range(2,4):
#     for j in range(i+1):
#         # Use '&' for element-wise comparison and wrap conditions in parentheses
#         elem_rows = ((result_df['bool_c'] == j) & (result_df['bool_t'] == i)).sum()
#         print(f'Total number of rows where predicted:{j} and total:{i} : {elem_rows}')


# equal_rows = (result_df['bool_c'] == result_df['bool_t']).sum()

# print(f'Total number of rows where values are equal: {equal_rows}')

# less_rows = (result_df['bool_c'] < result_df['bool_t']).sum()

# print(f'Total number of rows where values are less: {less_rows}')

# print(f'values in bool_c: {result_df["bool_c"].value_counts()}')

## Without NER

In [ ]:
# print(all_embeddings[0].shape)

In [ ]:
# print(input_df.columns)

In [ ]:
# top_k = 6  # Define how many top sentences you want to retrieve
# similarity_threshold = 0.05  # Define the similarity threshold

# result_data = []

# # Iterate over each row in input_df
# for i, row in input_df.iterrows():
#     # Step 1: Get the question embedding for the current row
#     question_embedding = question_embeddings[i].unsqueeze(0).cpu().numpy()
    
#     # Step 2: Get the sentence embeddings for the current row from the asr_df
#     # Assuming `all_embeddings` is the list of lists containing sentence embeddings for each row
#     sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
    
#     # Step 3: Get the corresponding sentence IDs for this row
#     sentence_ids = ast.literal_eval(asr_df['Sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings

#     # Step 4: Calculate cosine similarity between the question embedding and the sentence embeddings
#     similarity_scores = cosine_similarity(question_embedding, sentence_embeddings).flatten()
    
#     # Step 5: Filter based on similarity threshold
#     filtered_indices = [idx for idx, score in enumerate(similarity_scores) if score >= similarity_threshold]
    
#     # Step 6: Sort by similarity score and get top k sentences
#     top_indices = sorted(filtered_indices, key=lambda idx: similarity_scores[idx], reverse=True)[:top_k]
    
#     # Get the top k sentences and their corresponding sentence IDs
#     top_sentence_ids = [sentence_ids[idx] for idx in top_indices]  # Corresponding sentence IDs
#     top_sentences = [ast.literal_eval(asr_df['transcriptions'].iloc[i])[idx] for idx in top_indices]  # Sentences corresponding to top indices
#     pos_sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])
#     count = 0
#     for e1 in pos_sentence_ids:
#         if e1 in top_sentence_ids:
#             count+=1
# #     print(pos_sentence_ids.all() in top_sentence_ids)
#     # Step 7: Store the result for this row
#     result_data.append({
#         'q_id': row['QID'],
#         'top_sentence_ids': top_sentence_ids,
#         'top_sentences': top_sentences,
#         'similarity_scores': [similarity_scores[idx] for idx in top_indices],
#         'bool_c' : count,
#         'bool_t': len(pos_sentence_ids)
#     })

# # Optionally convert the result to a DataFrame
# result_df = pd.DataFrame(result_data)

# # Print the result
# print(result_df.head())



In [ ]:
# for i in range(2,4):
#     for j in range(i+1):
#         # Use '&' for element-wise comparison and wrap conditions in parentheses
#         elem_rows = ((result_df['bool_c'] == j) & (result_df['bool_t'] == i)).sum()

#         print(f'Total number of rows where predicted:{j} and total:{i} : {elem_rows}')


# equal_rows = (result_df['bool_c'] == result_df['bool_t']).sum()

# print(f'Total number of rows where values are equal: {equal_rows}')

# less_rows = (result_df['bool_c'] < result_df['bool_t']).sum()

# print(f'Total number of rows where values are less: {less_rows}')

# print(f'values in bool_c: {result_df["bool_c"].value_counts()}')

## NER first 

In [ ]:
entity_embeddings = model.encode(entity_df['Knowledge'].tolist(), convert_to_tensor=True, device=device)
knowledge_entity_ids = entity_df['Entity_ID'].tolist()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import ast

# top_k = 6  # Define how many top entities you want to retrieve
similarity_threshold = 0.05  # Define the similarity threshold

result_data = []

# Assume `knowledge_embeddings` contains embeddings of all entities
# Assume `knowledge_entity_ids` contains the IDs corresponding to the knowledge embeddings

for i, row in input_df.iterrows():
    # Step 1: Get the sentence embeddings for the current row from the asr_df
    sentence_embeddings = all_embeddings[i].cpu().numpy()  # Shape: (num_sentences_in_row, embedding_dim)
    
    # Step 2: Get the corresponding sentence IDs for this row
    sentence_ids = ast.literal_eval(asr_df['Sentence_ids'].iloc[i])  # List of sentence IDs corresponding to the embeddings
    
    # Step 3: Calculate cosine similarity between sentence embeddings and knowledge embeddings
    similarity_scores = cosine_similarity(sentence_embeddings,  entity_embeddings.cpu().numpy())  # Shape: (embedding_dim, num_sentences_in_row)
    
#     print(similarity_scores.shape)
    # Step 4: Process each sentence
    top_sentence_ids = []
    top_entities = []
    top_scores = []
    
    for j in range(len(sentence_ids)):
        # Get similarity scores for the current sentence
        sentence_similarity_scores = similarity_scores[j]
        
        # Filter based on similarity threshold
        filtered_indices = [idx for idx, score in enumerate(sentence_similarity_scores) if score >= similarity_threshold]
        
        # Sort by similarity score and get top k entities
        top_indice = sorted(filtered_indices, key=lambda idx: sentence_similarity_scores[idx], reverse=True)[0]
#         if i < 1: 
#             print(top_indice)
        # Get the top entities and their corresponding IDs
        top_entities_for_sentence = knowledge_entity_ids[top_indice]
#         top_scores_for_sentence = [sentence_similarity_scores[idx] for idx in top_indices]
        
        # Store results for this sentence
#         sentence_ids.append(sentence_ids[j])
        top_entities.append(top_entities_for_sentence)
#         top_scores.append(top_scores_for_sentence)
    
    # Flatten the results and count matches for positive entities
#     pos_sentence_ids = ast.literal_eval(input_df['positive_sentence_ids'].iloc[i])
#     count = sum(e1 in top_entity for top_entities_for_sentence in top_entities for e1 in pos_sentence_ids)
    
    # Step 7: Store the result for this row
    result_data.append({
        'q_id': row['QID'],
        'sentence_ids': sentence_ids,
        'top_entities': top_entities
#         'similarity_scores': top_scores,
#         'bool_c': count,
#         'bool_t': len(pos_sentence_ids)
    })

# Optionally convert the result to a DataFrame
result_df = pd.DataFrame(result_data)

# Print the result
print(result_df.head())


In [ ]:
result_df.to_csv("Task_3_asr_NER.csv")

# Full Knowledge

In [ ]:
# Step 2: Encode the input sentences and entity knowledge strings using GPU
input1_embeddings = model.encode(asr_df['Sentence_1_transcript'].tolist(), convert_to_tensor=True, device=device)
input2_embeddings = model.encode(asr_df['Sentence_2_transcript'].tolist(), convert_to_tensor=True, device=device)
entity_embeddings = model.encode(entity_df['Knowledge'].tolist(), convert_to_tensor=True, device=device)

In [ ]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1


# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50

print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")

In [ ]:
ner_df = pd.DataFrame(res_data)

In [ ]:
ner_df.to_csv("Task2_asr_entity_linked_CLIP_full_knowledge_results.csv")

# Partial Knowledge

In [ ]:
# Step 2: Encode the input sentences and entity knowledge strings using GPU
input1_embeddings = model.encode(asr_df['Sentence_1_transcript'].tolist(), convert_to_tensor=True, device=device)
input2_embeddings = model.encode(asr_df['Sentence_2_transcript'].tolist(), convert_to_tensor=True, device=device)

# Calculate the substring for the first 20% of each knowledge string
entity_df['Knowledge_20'] = entity_df['Knowledge'].apply(lambda x: x[:int(len(x) * 0.2)])

# Encode the first 20% of the knowledge strings
entity_embeddings = model.encode(entity_df['Knowledge_20'].tolist(), convert_to_tensor=True, device=device)


In [ ]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1


# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50

print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")

In [ ]:
ner_df = pd.DataFrame(res_data)

In [ ]:
ner_df.to_csv("Task2_asr_entity_linked_CLIP_partial_knowledge_results.csv")

# Labels Only

In [ ]:
# Step 2: Encode the input sentences and entity knowledge strings using GPU
input1_embeddings = model.encode(asr_df['Sentence_1_transcript'].tolist(), convert_to_tensor=True, device=device)
input2_embeddings = model.encode(asr_df['Sentence_2_transcript'].tolist(), convert_to_tensor=True, device=device)
entity_embeddings = model.encode(entity_df['Entity_name'].tolist(), convert_to_tensor=True, device=device)

In [ ]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1


# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50

print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")

In [ ]:
ner_df = pd.DataFrame(res_data)

In [ ]:
ner_df.to_csv("Task2_asr_entity_linked_CLIP_label_results.csv")